In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('questions.csv')

In [3]:
df.shape

(404351, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
new_df = df.sample(50000)

In [6]:
new_df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,1
is_duplicate,0


In [7]:
new_df.duplicated().sum()

np.int64(0)

In [8]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
257666,Can anyone find out the questioner when he or ...,How can I get my anonymity back on Quora?
212159,Is MBA in HR a good idea after doing B.tech if...,How is life after doing MBA in HR from SIBM?
289441,What is mapping?,What is a mapped drive?
356785,What should I know about moving from Pennsylva...,What is it like moving from Pennsylvania to So...
83991,What are the strongest majors in terms of job ...,What are the strongest majors in terms of job ...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

questions = list(ques_df['question1'].fillna('')) + list(ques_df['question2'].fillna(''))

cv = CountVectorizer(max_features = 3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [10]:
temp_df1 =  pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(50000, 6000)

In [11]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
257666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35741,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
386642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
130370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
temp_df['is_duplicate'] = new_df['is_duplicate']
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
257666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
83991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state=1)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7566

In [15]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.738